In [1]:
import pytest
import ipytest

ipytest.autoconfig()

In [2]:
def my_func(x):
    return x // 2 * 2 

# Execute all tests

In [3]:
# define the tests

def test_my_func():
    assert my_func(0) == 0
    assert my_func(1) == 0
    assert my_func(2) == 2
    assert my_func(3) == 2
    
    
@pytest.mark.parametrize('input,expected', [
    (0, 0),
    (1, 0),
    (2, 2),
    (3, 2),
])
def test_parametrized(input, expected):
    assert my_func(input) == expected
    
    
@pytest.fixture
def my_fixture():
    return 42
    
    
def test_fixture(my_fixture):
    assert my_fixture == 42

In [4]:
# execute the tests via pytest, arguments are passed to pytest
ipytest.run('-qq')

......                                                                                                                                                                                                                                  [100%]


# Execute only tests in the current cell

By removing any tests previously defined using `clean_tests`:

In [5]:
ipytest.clean_tests()

@pytest.fixture
def my_fixture():
    return 42
    
    
def test_fixture(my_fixture):
    assert my_fixture == 42
    
ipytest.run('-qq')

.                                                                                                                                                                                                                                       [100%]


By using ipytest's magics:

In [6]:
%%run_pytest[clean] -qq

def test_my_func():
    assert my_func(0) == 0
    assert my_func(1) == 0
    assert my_func(2) == 2
    assert my_func(3) == 2

.                                                                                                                                                                                                                                       [100%]


# Local rewriting

In [7]:
# below this line only asserts inside the test cell are rewritten
ipytest.config.rewrite_asserts = False

In [8]:
%%run_pytest[clean] -qq

def test_my_func():
    assert my_func(0) == 0
    assert my_func(1) == 0
    assert my_func(2) == 2
    assert my_func(3) == 2

.                                                                                                                                                                                                                                       [100%]


In [9]:
%%run_pytest[clean] -qq

def test_my_func():
    assert my_func(0) == 1

F                                                                                                                                                                                                                                       [100%]
================================================================================================================== FAILURES ===================================================================================================================
________________________________________________________________________________________________________________ test_my_func _________________________________________________________________________________________________________________

    def test_my_func():
>       assert my_func(0) == 1
E       assert 0 == 1
E        +  where 0 = my_func(0)

<ipython-input-9-aeb816b13d1b>:2: AssertionError
=========================================================================================================== short test s

In [10]:
# NOTE: no magics, therefore no assertion rewriting will be performed

def test_my_func():
    assert my_func(0) == 1
    
ipytest.run()

F                                                                                                                                                                                                                                       [100%]
================================================================================================================== FAILURES ===================================================================================================================
________________________________________________________________________________________________________________ test_my_func _________________________________________________________________________________________________________________

    def test_my_func():
>       assert my_func(0) == 1
E       AssertionError

<ipython-input-10-76fa61b38cca>:4: AssertionError
=========================================================================================================== short test summary info ===================